In [0]:
from pyspark.sql.functions import *

In [0]:
load_path = "abfss://raw@hisadls.dfs.core.windows.net/"
checkpoint_path = "abfss://checkpoint@hisadls.dfs.core.windows.net/"

raw_df = spark.read.option("multiline", "true")\
    .json(load_path)

raw_df.printSchema()

root
 |-- IsInvasive: boolean (nullable = true)
 |-- IsPatientProvided: boolean (nullable = true)
 |-- alias: string (nullable = true)
 |-- category: string (nullable = true)
 |-- choices: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- laytext: string (nullable = true)
 |    |    |-- relatedanswertag: string (nullable = true)
 |    |    |-- text: string (nullable = true)
 |    |    |-- value: long (nullable = true)
 |-- default: double (nullable = true)
 |-- laytext: string (nullable = true)
 |-- max: double (nullable = true)
 |-- min: double (nullable = true)
 |-- name: string (nullable = true)
 |-- step: double (nullable = true)
 |-- subcategory1: string (nullable = true)
 |-- subcategory2: string (nullable = true)
 |-- subcategory3: string (nullable = true)
 |-- subcategory4: string (nullable = true)
 |-- text: string (nullable = true)
 |-- type: string (nullable = true)
 |-- wiki: string (nullable = true)
 |-- wiki2: string (nullable = true)


In [0]:
schema = raw_df.schema

bronze_df = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "json")\
        .option("multiline", "true")\
            .schema(schema)\
                .load(load_path)

In [0]:
bronze_df.writeStream.format("delta")\
    .option("checkpointLocation", checkpoint_path)\
        .outputMode("append")\
            .trigger(once=True)\
                .table("surakshadb.bronze_stg.symptoms_raw")

In [0]:
# df = spark.read.table("surakshadb.bronze_stg.symptoms_raw")
# df.limit(5).display()

IsInvasive,IsPatientProvided,alias,category,choices,default,laytext,max,min,name,step,subcategory1,subcategory2,subcategory3,subcategory4,text,type,wiki,wiki2,wiki3,wiki4
false,true,,Constitutional and vital signs physical examination,null,35.0,Age,120.0,18.0,Age,null,,,,,What is the age?,integer,,,,
null,false,"weight, height",Constitutional and vital signs physical examination,null,19.0,"What is your BMI (body mass index, calculate by either of these formulas BMI = 703 ? weight (lbs) / [height (in)]2 or BMI = weight (kg) / [height (m)]2)",60.0,11.0,BMI,0.1,,,,,What is the BMI?,double,,,,
null,false,,Constitutional and vital signs physical examination,null,98.0,"What is the highest recorded temperature (using the ear thermometer)? Hint: the ear temperature is usually 1F higher than temperature measured on forehead or armpit temperature, so to calculate the ear temperature add 1F to the temperature measured on forehead or in armpit)",109.0,95.0,Temp,0.1,,,,,"What is the highest tympanic (ear) temperature measured (Fahrenheit), which is usually higher by 1F than axilar or forehead temperature?",double,Temp website,Temp website 2,Temp website 3,Temp website 4
null,false,,Constitutional and vital signs physical examination,null,70.0,What is your current pulse rate (Hint: the pulse rate is the count of pulse beats within the minute)?,220.0,0.0,HeartRate,null,,,,,What is the pulse rate (not always equals the heart rate)?,integer,,,,
null,false,"systolic, sbp",Constitutional and vital signs physical examination,null,115.0,"What is your current systolic blood pressure? (Hint: the top number from your blood pressure reading is called the ""systolic blood pressure"", for example if your meter says ""120/80 mmHg"", the systolic blood pressure is ""120"")",300.0,0.0,SBP,null,,,,,What is the systolic blood pressure?,integer,,,,
